In [59]:
from dotenv import load_dotenv
import json
from openai import OpenAI
import os
load_dotenv()

True

In [43]:
client = OpenAI(
    api_key = os.getenv("RUNPOD_TOKEN"),
    base_url = os.getenv("RUNPOD_BASE_URL")
)
model_name = os.getenv("MODEL_NAME")

In [48]:
messages=[
        {"role": "user", "content": "Why is the capital of Egypt?"}
]


# get llama response

In [49]:
def get_chatbot_response(client, model_name, messages, temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})
    # Create a chat completion
    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        max_tokens=2000,
    )
    return response.choices[0].message.content

In [ ]:
get_chatbot_response(client, model_name, messages)

# Prompt engineering
## Structured output 

In [51]:
system_prompt = """
You are a helper assistant that answer questions about crispr cas9.
Your output should be in a structured json format exactly like the one the below. You are not allowed to write anything other than json object:
[
    {
        "question": the question that you will answer
        "answer": concise answer for the question
    }
]
"""

In [53]:
messages=[
        {"role": "system", "content": system_prompt}
]

messages.append({"role": "user", "content": "How to design a crispr-cas9 experiment for gene editing?"})

In [ ]:
response = get_chatbot_response(client, model_name, messages)

In [61]:
json_response = json.loads(response)
print(json_response)

[{'question': 'How to design a CRISPR-Cas9 experiment for gene editing?', 'answer': {'steps': ['1. Identify the target gene and its function', '2. Design a guide RNA (gRNA) that specifically binds to the target gene', '3. Choose a suitable Cas9 enzyme and its variant (e.g., SpCas9, AsCas9)', '4. Optimize the gRNA-Cas9 complex design for efficient editing', '5. Select a suitable cell type or organism for editing', '6. Validate the gRNA-Cas9 complex using in vitro assays (e.g., PCR, sequencing)', '7. Perform the CRISPR-Cas9 editing experiment using the optimized gRNA-Cas9 complex', '8. Verify the editing efficiency and specificity using molecular biology techniques (e.g., PCR, sequencing, Western blot)', '9. Analyze the edited cells or organisms for desired phenotypes or functions']}}]


## Input structuring

In [64]:
user_input = """
Get me basic information on different crispr systems:
    ```
        1. cas9
        2. cas12a
        3. prime editor
        4. base editor
    ```
"""
messages=[
        {"role": "system", "content": system_prompt}
]
messages.append({"role": "user", "content": user_input})
response = get_chatbot_response(client, model_name, messages)

In [65]:
response

'[\n    {\n        "question": "What is CRISPR-Cas9?",\n        "answer": "CRISPR-Cas9 is a bacterial immune system that uses a small RNA guide to locate and cleave specific DNA sequences, allowing for precise genome editing."\n    },\n    {\n        "question": "What is CRISPR-Cas12a?",\n        "answer": "CRISPR-Cas12a is a type of CRISPR system that uses a smaller RNA guide and can edit genes without leaving a footprint, making it more precise and efficient than CRISPR-Cas9."\n    },\n    {\n        "question": "What is the Prime Editor?",\n        "answer": "The Prime Editor is a CRISPR-based system that can make precise, high-fidelity edits to the genome by using a combination of DNA repair pathways and a novel RNA guide."\n    },\n    {\n        "question": "What is the Base Editor?",\n        "answer": "The Base Editor is a CRISPR-based system that can make precise, single-base edits to the genome by using a combination of DNA repair pathways and a novel RNA guide, allowing for 